In [2]:
import pandas as pd
import numpy as np
import math
import os
import seaborn as sns
import matplotlib.pyplot as plt
import concurrent.futures #Para paralelizar

import warnings
warnings.filterwarnings('ignore')

In [3]:
#df = pd.read_csv('/home/ferambriz/Programs/ncov_screening/results/custom-build/aligned.fasta')
df = pd.read_csv('/home/ferambriz/Programs/ncov/results/custom-build/aligned.fasta')
df.head()

,>otter/USA/GA-21-011485-003/2021
0,-------------------------------------AACTTTCGA...
1,>otter/USA/GA-21-011485-011/2021
2,ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGA...
3,>otter/USA/GA-21-011485-009/2021
4,ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGA...


In [4]:
#============================== Pasa de un tabla de una columna a n caracteres ===========================#
def extract_alignments_UTR(df, caracter):
    headers = list(df.columns)
    for i in df[headers[0]]:
        if i.startswith(">"):
            headers.append(i)
    df = df[~df[headers[0]].str.startswith('>')]
    df['headers'] = headers
    

    df = df[headers[0]].apply(list).apply(pd.Series)

    # Opcionalmente, puedes renombrar las columnas
    df.columns = [f'caracter_{i+1}' for i in range(df.shape[1])]
    df.insert(0, "ID", headers)
    
    df = df[(df[caracter]!= '-') & (df[caracter]!='N')]
    return df
#extract_alignments_UTR(df, 'caracter_20').head()

In [5]:
#===========================  Extracción de caracteres ========================#
def read_parts_df(df, nsamples, start, end, caracter):
    df_i = df.iloc[start:end]
    df_i = extract_alignments_UTR(df_i, caracter)
    return df_i
#================================ Bloques de nsamples ===============================#
def read_parts_df_global(df, nsamples, caracter):
    ranges_lst = list(range (0, len(df) + nsamples*2, (nsamples*2)))
    df_bd = pd.DataFrame()

    for i in range(len(ranges_lst)-1):
        print(ranges_lst[i], ranges_lst[i+1]-1)
        df_i = read_parts_df(df, nsamples, ranges_lst[i], ranges_lst[i+1]-1, caracter)
        df_bd = pd.concat([df_bd, df_i])
    
    return df_bd
#read_parts_df_global(df, 2000,'caracter_20')

In [6]:
def len_filter_UTR (start, end, nsamples):
    lst = []
    for i in range(start, end+1):
        print('caracter_' + str(i))
        df_i = read_parts_df_global(df, nsamples,'caracter_' + str(i))
        print(len(df_i))
        lst.append(len(df_i))
    
    df_final = pd.DataFrame({
        'site': list(range(start, end+1)),
        'Nsamples': lst
    })
    return df_final

Consideraciones
* 5'UTR -> start: 1 ; end: 265
* 3'UTR -> start: 29675 ; end: 29903

In [7]:
df_5UTR = read_parts_df_global(df, 10000, 'caracter_1')
df_5UTR

0 19999
20000 39999
40000 59999
60000 79999
80000 99999
100000 119999
120000 139999
140000 159999
160000 179999
180000 199999
200000 219999
220000 239999
240000 259999
260000 279999
280000 299999
300000 319999
320000 339999
340000 359999
360000 379999
380000 399999
400000 419999
420000 439999
440000 459999
460000 479999
480000 499999
500000 519999
520000 539999
540000 559999
560000 579999
580000 599999
600000 619999
620000 639999
640000 659999
660000 679999
680000 699999
700000 719999
720000 739999
740000 759999
760000 779999
780000 799999
800000 819999
820000 839999
840000 859999
860000 879999
880000 899999
900000 919999


,ID,caracter_1,caracter_2,caracter_3,caracter_4,caracter_5,caracter_6,caracter_7,caracter_8,caracter_9,...,caracter_29894,caracter_29895,caracter_29896,caracter_29897,caracter_29898,caracter_29899,caracter_29900,caracter_29901,caracter_29902,caracter_29903
2,>otter/USA/GA-21-011485-011/2021,A,T,T,A,A,A,G,G,T,...,-,-,-,-,-,-,-,-,-,-
4,>otter/USA/GA-21-011485-009/2021,A,T,T,A,A,A,G,G,T,...,-,-,-,-,-,-,-,-,-,-
6,>otter/USA/GA-21-011485-007/2021,A,T,T,A,A,A,G,G,T,...,-,-,-,-,-,-,-,-,-,-
8,>otter/USA/FL-21-025190-002/2021,A,T,T,A,A,A,G,G,T,...,-,-,-,-,-,-,-,-,-,-
12,>otter/USA/FL-21-025190-001/2021,A,T,T,A,A,A,G,G,T,...,-,-,-,-,-,-,-,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
912446,>Wuhan-Hu-1/2019,A,T,T,A,A,A,G,G,T,...,A,A,A,A,A,A,A,A,A,A
912536,>USA/TX-CDC-STM-KWDXKWHMH/2022,A,T,T,A,A,A,G,G,T,...,A,A,A,A,A,A,A,A,A,A
912548,>PHL/210316-2/2021,A,T,T,A,A,A,G,G,T,...,A,A,A,A,A,A,A,A,A,A
912552,>ARG/210316-1/2021,A,T,T,A,A,A,G,G,T,...,A,A,A,A,A,A,A,A,A,A


In [9]:
df_5UTR['ID'].to_csv('Final_samples.tsv', sep = '\t', index = False)

In [ ]:
df_3UTR = len_filter_UTR(29675, 29903, 500, 16)

# Restructuración para optimización de RAM

In [3]:
#============================== Pasa de un tabla de una columna a n caracteres ===========================#
def extract_alignments_UTR(df, caracter):
    headers = list(df.columns)
    for i in df[headers[0]]:
        if i.startswith(">"):
            headers.append(i)
    df = df[~df[headers[0]].str.startswith('>')]
    df['headers'] = headers
    

    df = df[headers[0]].apply(list).apply(pd.Series)

    # Opcionalmente, puedes renombrar las columnas
    df.columns = [f'caracter_{i+1}' for i in range(df.shape[1])]
    df.insert(0, "ID", headers)
    
    df = df[(df[caracter]!= '-') & (df[caracter]!='N')]
    return df
#extract_alignments_UTR(df, 'caracter_20').head()

In [4]:
#===========================  Extracción de caracteres ========================#
def read_parts_df(df, nsamples, start, end, caracter):
    df_i = df.iloc[start:end]
    df_i = extract_alignments_UTR(df_i, caracter)
    return len(df_i)
#================================ Bloques de nsamples ===============================#
def read_parts_df_global(df, nsamples, caracter):
    ranges_lst = list(range (0, len(df) + nsamples*2, (nsamples*2)))
    df_bd = pd.DataFrame()
    
    len_total = 0
    for i in range(len(ranges_lst)-1):
        print(ranges_lst[i], ranges_lst[i+1]-1)
        len_i = read_parts_df(df, nsamples, ranges_lst[i], ranges_lst[i+1]-1, caracter)
        len_total = len_i + len_total
        print(len_total)
    
    return len_total
#read_parts_df_global(df, 2000,'caracter_20')

In [5]:
def len_filter_UTR (start, end, nsamples):
    lst = []
    for i in range(start, end+1):
        print('caracter_' + str(i))
        len_i = read_parts_df_global(df, nsamples,'caracter_' + str(i))
        lst.append(len(len_i))
    
    df_final = pd.DataFrame({
        'site': list(range(start, end+1)),
        'Nsamples': lst
    })
    return df_final

In [8]:
df_5UTR = len_filter_UTR(1, 2, 10000)
df_5UTR

caracter_1
0 19999
170
20000 39999
733
40000 59999
1670
60000 79999
2848
80000 99999
4185
100000 119999
4670
120000 139999
4700
140000 159999
5292
160000 179999
5677
180000 199999
6728
200000 219999


KeyboardInterrupt: 